In [1]:
import cv2
import numpy as np 
import os

In [2]:
# !pip install opencv-python

In [3]:
def process_images(folder_path, target_size=(400, 400)):
    gray_images = []  # Initialize an empty list to store grayscale images
    
    # Check if the directory exists
    if not os.path.exists(folder_path):
        print(f"Directory does not exist: {folder_path}")
        return gray_images

    # Iterate through all files in the directory
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            resized_gray_image = cv2.resize(gray_image, target_size)
            gray_images.append(resized_gray_image)  # Append the resized grayscale image to the list
            
    # Recursively process images in subdirectories
    for subdir in os.listdir(folder_path): 
        subfolder_path = os.path.join(folder_path, subdir)
        
        if os.path.isdir(subfolder_path):
            gray_images.extend(process_images(subfolder_path, target_size))  # Extend the list with grayscale images from subdirectories
    
    return gray_images  # Return the list of grayscale images

# Path to the main directory containing the subdirectories with images
main_path = "The IQ-OTHNCCD lung cancer dataset"
gray_images = []

# Process images from each subdirectory within the main directory
subdirectories = ["Bengin cases", "Malignant cases", "Normal cases"]
for subdirectory in subdirectories:
    path = os.path.join(main_path, subdirectory)
    if os.path.exists(path):
        gray_images.extend(process_images(path))
    else:
        print(f"Subdirectory does not exist: {path}")



In [4]:
# Convert the list of images to a NumPy array
gray_arr = np.array(gray_images)

# Create a new array to store the flattened images
flattened_images = np.empty((len(gray_arr), gray_arr[0].size))

# Flatten each image in the array
for i in range(len(gray_arr)):
    flattened_images[i] = gray_arr[i].flatten()

# Check the shape of the array after flattening
print(flattened_images.shape)

# Create the label array
y = np.zeros(1097)
y[0:119] = 2
y[120:680] = 1
y[681:1096] = 0

print(y)

(1097, 160000)
[2. 2. 2. ... 0. 0. 0.]


In [5]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
pca=PCA(n_components=120)
pca.fit (flattened_images)
Pca=pca.fit_transform(flattened_images)
Pca.shape
# # Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(Pca,y, test_size=0.1, random_state=42)




In [6]:
from sklearn.svm import SVC
svc = SVC(kernel='poly', C=1.0, random_state=21)
svc.fit( X_train, y_train)


SVC(kernel='poly', random_state=21)

In [7]:
# # Predict the labels for the test data
y_pred = svc.predict(X_test)

# # Evaluate the performance of the classifier
accuracy = svc.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.9636363636363636


In [8]:
def detect_face(real_image_path):
    real_image = cv2.imread(real_image_path, cv2.IMREAD_GRAYSCALE)
    
    resized_gray_image = cv2.resize(real_image, (400,400))

    # Flatten the real image
    flattened_real_image = resized_gray_image.flatten()
    # print(flattened_real_image.shape)
    # Apply PCA transformation
    pca_real_image = pca.transform([flattened_real_image])
    #rint(pca_real_image)

    # Predict using the trained SVM classifier
    predicted_label = svc.predict(pca_real_image)
    if predicted_label[0]==0:
        print("normal")
    elif predicted_label[0]==1:
        print("Malignant")
    elif predicted_label[0]==2:
        print("Bengin")    
    
image_path=["Normal case (12).jpg","000020_04_01_159.png","Bengin case (13).jpg"]
for i in range(len(image_path)):
    detect_face(image_path[i])


normal
normal
Bengin


In [9]:
import joblib

# Assuming `model` is your trained scikit-learn model
joblib.dump(svc, 'my_model.pkl')
joblib.dump(Pca," Pca_model.pkl")

[' Pca_model.pkl']

In [ ]:
! 